In [10]:
import pandas as pd
from difflib import SequenceMatcher

In [11]:
with open('words.txt', 'r', encoding='utf-8') as f:
    hotspot_words = f.read().split('\n')
hotspot_words

['celebration',
 'dinner',
 'affair',
 'amusement',
 'ball',
 'banquet',
 'barbecue',
 'bash',
 'blowout',
 'carousal',
 'carousing',
 'cocktails',
 'diversion',
 'do',
 'entertainment',
 'feast',
 'festivity',
 'fun',
 'function',
 'gala',
 'luncheon',
 'orgy',
 'prom',
 'reception',
 'riot',
 'shindig',
 'social',
 'soiree',
 'splurge',
 'spree',
 'festive',
 'occasion',
 'get-together',
 'feast',
 'anniversary',
 'commemoration',
 'competition',
 'fair',
 'feast',
 'gala',
 'holiday',
 'carnival',
 'entertainment',
 'festivities',
 'fete',
 'fiesta',
 'jubilee',
 'merrymaking',
 'treat',
 'field day',
 'association',
 'caucus',
 'collection',
 'conclave',
 'conference',
 'congregation',
 'convention',
 'crowd',
 'function',
 'group',
 'mass',
 'meeting',
 'party',
 'throng',
 'acquisition',
 'affair',
 'aggregate',
 'aggregation',
 'band',
 'body',
 'bunch',
 'clambake',
 'company',
 'concentration',
 'concourse',
 'congress',
 'convocation',
 'crush',
 'drove',
 'flock',
 'gain',
 

In [12]:
df = pd.read_csv('all.csv', header=0)

In [13]:
def hotspot_found(text):
    words = text.split()
    for hotspot_word in hotspot_words:
        for word in words:
            if SequenceMatcher(None, word.lower(), hotspot_word.lower()).ratio() >= 0.8:
                return True
    return False

In [14]:
hotspots_tweets_mask = df.translated_text.apply(hotspot_found)

In [16]:
hotspots_tweets_mask.value_counts()

True     31336
False    20347
Name: translated_text, dtype: int64

In [20]:
for x in df[hotspots_tweets_mask].translated_text[:1000]:
    print(x)
    print('------------------------------------')

Honorable professors of law at the Lebanese University respectedPublic consultation:What kind of legal liability does one or more students contract with the # Corona virus as a result of insisting that the exams be conducted in attendance despite warnings and appeals?Death Exams
------------------------------------
The "Lebanese University Students Association" revealed on Twitter a case of # Corona in the ranks of students at the Faculty of Health - Hadath Complex.
------------------------------------
#UrgentThe director of the Prisoners' Information Office Nahid Al-Fakhouri: We previously warned against the arrival of # Corona Virus in prisons and what we have data foreshadows much worse.
------------------------------------
#UrgentThe director of the Prisoners' Information Office Nahid Al-Fakhouri: We previously warned against the arrival of # Corona Virus in prisons and what we have data foreshadows much worse.
------------------------------------
Director of the Prisoner's Informa

In [27]:
df[hotspots_tweets_mask].to_csv('hotspots_tweets_v1.csv', index=False)

In [28]:
df = pd.read_csv('hotspots_tweets_v1.csv', header=0)